In [1]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torchvision
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils
from torchvision import transforms
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from Data_Retriever import ImageData
from Resnet_UNet import ResNetUNet
from Metrics import Metrics
import time
import os

In [2]:
CUDA_LAUNCH_BLOCKING="1"

In [3]:
tr = pd.read_csv('train.csv')
tr = tr[tr['ClassId'] == 1]

In [4]:
train_df, test_df = train_test_split(tr, test_size=0.33, random_state=123)

In [5]:
len(train_df)

600

In [6]:
train_transforms = transforms.Compose([
                                transforms.Resize((256, 256)),
                                transforms.ToTensor()])
                                #transforms.Normalize([0.485, 0.456, 0.406],
                                                     #[0.229, 0.224, 0.225])])
train_data = ImageData(df = train_df, transform = train_transforms, subset="train")
train_loader = DataLoader(dataset = train_data, batch_size=2)

In [7]:
test_transforms = transforms.Compose([
                                transforms.Resize((256, 256)),
                                transforms.ToTensor()])
                                #transforms.Normalize([0.485, 0.456, 0.406],
                                                     #[0.229, 0.224, 0.225])])
test_data = ImageData(df = test_df, transform = test_transforms, subset="test")
test_loader = DataLoader(dataset = train_data, batch_size=2)

In [8]:
torch.manual_seed(42)
model = ResNetUNet(n_class=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), weight_decay=1e-4, lr = 0.001, momentum=0.9)

In [9]:
print(model)

ResNetUNet(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running

In [10]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

In [11]:
metrics = Metrics()

In [12]:
%%time
EPOCHS = 5
epochs = 0
start_time = time.time()

#configure the model for GPU training
model = model.to(DEVICE)
train_losses = []
test_losses = []
train_dice_loss = []
test_dice_loss = []
train_dice_coeff = []
test_dice_coeff = []

PATH = 'segmentation_1_checkpoint.pth'

if os.path.exists(PATH):
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epochs = checkpoint['epoch']
    train_losses = checkpoint['train_loss']
    test_losses = checkpoint['test_loss']
    train_dice_loss = checkpoint['train_dice_loss']
    test_dice_loss = checkpoint['test_dice_loss']
    train_dice_coeff = checkpoint['train_dice_coeff']
    test_dice_coeff = checkpoint['test_dice_coeff']
    
    
for i in range(epochs, EPOCHS):     
    trn_corr = 0
    tst_corr = 0
    model.train()         
    for b, (data, target) in enumerate(train_loader): 
        b += 1
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)  
        loss = criterion(output, target)
        dice_loss = metrics.diceloss(output, target)
        dice_coeff = metrics.dice_coeff(output, target)
        loss.backward()
        optimizer.step()
        
    print(f'Model training')
    print(f'epoch: {i:2} loss: {loss.item():5.8f} \
    dice_loss:{dice_loss.item():5.2f}  dice_coeff{dice_coeff.item():10.3}')  
            

    train_losses.append(loss)
    train_dice_loss.append(dice_loss)
    train_dice_coeff.append(dice_coeff)
    
     # Run the testing batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            b+=1
            
            X_test = X_test.to(DEVICE)
            y_test = y_test.to(DEVICE)
            # Apply the model
            y_val = model(X_test)


        loss = criterion(y_val, y_test)
        dice_loss = metrics.diceloss(y_val, y_test)
        dice_coeff = metrics.dice_coeff(y_val, y_test)
        test_losses.append(loss)
        test_dice_loss.append(dice_loss)
        test_dice_coeff.append(dice_coeff)

        print(f'Model Validation')
        print(f'epoch: {i:2} loss: {loss.item():5.8f} \
        dice_loss:{dice_loss.item():5.2f}  dice_coeff{dice_coeff.item():10.3}')  
    
    checkpoint = {'epoch': i,
                  'state_dict': model.state_dict(),
                  'optimizer' : optimizer.state_dict(),
                  'train_loss' : train_losses,
                  'test_loss' : test_losses,
                  'train_dice_loss' : train_dice_loss,
                  'test_dice_loss' : test_dice_loss,
                  'train_dice_coeff':train_dice_coeff,
                  'test_dice_coeff':test_dice_coeff}

    torch.save(checkpoint, PATH)

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed


C:\Users\SIDDHARTA\anaconda3\envs\pytorchenv\lib\site-packages\torch\nn\functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Model training
epoch:  0 loss: 0.00141796     dice_loss: 0.99  dice_coeff  8.57e-05
Model Validation
epoch:  0 loss: 0.00141300         dice_loss: 0.99  dice_coeff  8.57e-05
Model training
epoch:  1 loss: 0.00096414     dice_loss: 0.99  dice_coeff  8.71e-05
Model Validation
epoch:  1 loss: 0.00096329         dice_loss: 0.99  dice_coeff  8.71e-05
Model training
epoch:  2 loss: 0.00084090     dice_loss: 0.98  dice_coeff  8.63e-05
Model Validation
epoch:  2 loss: 0.00084051         dice_loss: 0.98  dice_coeff  8.63e-05
Model training
epoch:  3 loss: 0.00079373     dice_loss: 0.98  dice_coeff  8.61e-05
Model Validation
epoch:  3 loss: 0.00079348         dice_loss: 0.98  dice_coeff  8.61e-05
Model training
epoch:  4 loss: 0.00076791     dice_loss: 0.97  dice_coeff  8.59e-05
Model Validation
epoch:  4 loss: 0.00076775         dice_loss: 0.97  dice_coeff  8.59e-05

Duration: 7591 seconds
Wall time: 2h 6min 31s


In [13]:
torch.save(model.state_dict(), 'Segmentation_1.pt')

In [12]:
model.to(DEVICE)
model.load_state_dict(torch.load('Segmentation_1.pt'));